In [4]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [5]:
password = 'PASSWORD' #Insert your password
db_name = 'zams'

In [6]:
conn = pymysql.connect(host='127.0.0.1', 
                       port=3306, 
                       user='root', 
                       password=password, 
                       database=db_name)

In [7]:
#Connecting to MySQL database
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

Note that result for all queries are for the first quarter of 2024

In [8]:
query1 = """
SELECT 
    d.Depart_Name,
    round(MAX(emp_avg_salaries.avg_salary), 2) AS Max_Average_Salary,
    round(AVG(s.Gross_Pay),2) AS Department_Avg_Salary
FROM zams.salaries s
Left JOIN zams.employees e ON s.Employee_ID = e.Employee_ID
Left JOIN zams.departments d ON e.Depart_ID = d.Depart_ID
LEFT JOIN (
    SELECT 
        e.Employee_ID,
        AVG(s.Gross_Pay) AS avg_salary
    FROM zams.salaries s
    JOIN zams.employees e ON s.Employee_ID = e.Employee_ID
    GROUP BY e.Employee_ID
) AS emp_avg_salaries ON e.Employee_ID = emp_avg_salaries.Employee_ID

GROUP BY d.Depart_Name;
"""
query1 = pd.read_sql(query1, con=connection)

In [9]:
query1 #note this is for the first quarter

,Depart_Name,Max_Average_Salary,Department_Avg_Salary
0,Communications,11600.00,10450.55
1,Engineering and Technology,12772.22,11497.92
2,Legal,12827.78,11377.08
3,Finance,18500.00,13505.36
4,Marketing,13241.66,11534.03
5,Service and Support,13533.33,10557.41
6,Design,15266.67,12447.92
7,Sales,15125.00,11654.17
8,Facilities,13177.78,10619.68


In [10]:
query2 = """
SELECT 
    d.Depart_Name AS DepartmentName,
    SUM(s.Quarterly_Bonus) AS SumOfBonuses,
    SUM(s.Gross_Pay) AS SumOfGrossSalaries,
    d.Budget_millions,
    ROUND(((SUM(s.Quarterly_Bonus) + SUM(s.Gross_Pay)) / (d.Budget_millions*1000000)), 4) * 100 AS PercentageOfBudgetToSalaries
FROM 
    Employees e
JOIN 
    Salaries s ON e.Employee_ID = s.Employee_ID
JOIN 
    Departments d ON e.Depart_ID = d.Depart_ID
GROUP BY 
    d.Depart_Name, d.Budget_millions;

"""
query2 = pd.read_sql(query2, con=connection)

In [11]:
query2

,DepartmentName,SumOfBonuses,SumOfGrossSalaries,Budget_millions,PercentageOfBudgetToSalaries
0,Communications,82000.0,156758.32,7,3.41
1,Engineering and Technology,72000.0,137974.99,25,0.84
2,Legal,105000.0,204787.49,6,5.16
3,Finance,148500.0,283612.50,15,2.88
4,Marketing,109500.0,207612.50,10,3.17
5,Service and Support,100000.0,190033.34,8,3.63
6,Design,114000.0,224062.52,9,3.76
7,Sales,73500.0,139850.01,12,1.78
8,Facilities,102500.0,191154.16,14,2.10


In [12]:
query3 = """
SELECT 
    e.First_name AS FirstName,
    e.Last_name AS LastName,
    d.Depart_name AS DepartmentName,
    s.Gross_pay AS GrossPay

FROM 
    Employees e

JOIN
    Departments d ON e.Depart_ID = d.Depart_ID
    
JOIN 
    Salaries s ON e.Employee_ID = s.Employee_ID
    
JOIN
    (
        SELECT 
            Depart_ID,
            MAX(Gross_pay) AS MaxGrossPay
        FROM 
            Employees e
        JOIN 
            Salaries s ON e.Employee_ID = s.Employee_ID
        WHERE 
            s.Salary_ID > 100
        GROUP BY 
            Depart_ID
    ) max_salaries ON d.Depart_ID = max_salaries.Depart_ID AND s.Gross_pay = max_salaries.MaxGrossPay
    
WHERE 
    s.Salary_ID > 100

GROUP BY 
    e.First_name, e.Last_name, d.Depart_name, s.Gross_pay

ORDER BY 
    s.Gross_pay DESC;

"""

query3 = pd.read_sql(query3, con=connection)

In [13]:
query3

,FirstName,LastName,DepartmentName,GrossPay
0,Joshua,Beck,Finance,20583.33
1,Sarah,Mendez,Facilities,17466.67
2,Jennifer,Gibson,Design,15466.67
3,Steven,Barrera,Service and Support,14500.00
4,Ashley,Riley,Sales,14500.00
5,Edward,Austin,Engineering and Technology,14191.67
6,ikun,Cai,Marketing,13533.33
7,Barbara,Adams,Legal,12350.00
8,Terri,Carter,Communications,9258.33


In [14]:
query4 = """
SELECT 
e.Employee_ID,
e.First_Name, 
e.Last_Name,
d.Depart_Name AS Department,
AVG(s.Gross_Pay) AS Avg_Gross_Wage,
AVG(s.Tax_Deduction) AS Avg_Tax_Paid,
AVG(s.Net_Pay) AS Avg_Net_Wage

FROM salaries AS s

LEFT JOIN employees AS e
    ON s.Employee_ID = e.Employee_ID
    
LEFT JOIN departments AS d
    ON e.Depart_ID = d.Depart_ID
    
GROUP BY e.Employee_ID, e.First_Name, e.Last_Name, d.Depart_Name;
    
"""
query4 = pd.read_sql(query4, con=connection)

In [15]:
query4

,Employee_ID,First_Name,Last_Name,Department,Avg_Gross_Wage,Avg_Tax_Paid,Avg_Net_Wage
0,14,Daniel,Johnson,Communications,11100.000000,2730.600000,7681.200000
1,11,Mary,Hopkins,Engineering and Technology,9801.386667,1764.250000,7429.456667
2,10,Joshua,King,Legal,8644.443333,1780.756667,6327.733333
3,15,Gary,Simon,Finance,13541.670000,3656.253333,9045.830000
4,16,Michael,Murray,Marketing,13241.663333,3787.113333,8633.563333
5,17,Melissa,Wright,Service and Support,6900.000000,1276.500000,5195.700000
6,13,Terri,Carter,Communications,10174.996667,3785.100000,5759.053333
7,18,Ronald,Gutierrez,Design,8738.890000,2132.290000,6064.790000
8,12,Jennifer,Vasquez,Legal,12827.776667,2244.860000,9787.593333
9,19,Stephanie,Porter,Design,12141.670000,3253.970000,8134.920000


In [16]:
query5 = """
SELECT 
    d.Depart_Name,
    COUNT(DISTINCT CASE WHEN s.Total_Work_Time > 160 THEN s.Employee_ID END) AS Employees_Working_Overtime,
    COUNT(DISTINCT e.Employee_ID) AS Total_Employees
FROM 
    zams.salaries s
LEFT JOIN 
    zams.employees e ON s.Employee_ID = e.Employee_ID
LEFT JOIN 
    zams.departments d ON e.Depart_ID = d.Depart_ID
GROUP BY 
    d.Depart_Name;

"""
query5 = pd.read_sql(query5, con=connection)

In [17]:
query5

,Depart_Name,Employees_Working_Overtime,Total_Employees
0,Communications,1,5
1,Design,4,6
2,Engineering and Technology,3,4
3,Facilities,3,6
4,Finance,5,7
5,Legal,5,6
6,Marketing,3,6
7,Sales,2,4
8,Service and Support,3,6


In [18]:
query6 = """
SELECT 
    e.First_Name AS Employee_Name,
    MONTH(e.Birth_Date) AS Birth_Month,
    DAY(e.Birth_Date) AS Birth_Day,
    d.Depart_Name AS Department
FROM 
    Employees AS e
JOIN 
    Departments AS d
ON 
    e.Depart_ID = d.Depart_ID
WHERE 
    MONTH(e.Birth_Date)> 6
ORDER BY Birth_Month ASC, Birth_Day ASC
"""
query6 = pd.read_sql(query6, con=connection)

In [19]:
query6

,Employee_Name,Birth_Month,Birth_Day,Department
0,Amanda,7,23,Service and Support
1,Michael,7,24,Marketing
2,ikun,7,30,Marketing
3,Nicole,7,31,Legal
4,Joshua,8,31,Legal
5,Dennis,9,3,Sales
6,Samantha,9,3,Finance
7,Monica,9,7,Facilities
8,Michael,9,24,Communications
9,Megan,9,24,Service and Support
